### dGen Post-Run Analysis (Baseline vs Policy)
 - Loads per-state CSVs (baseline/policy)
 - Computes portfolio & cumulative bill savings (cohort carry-forward via `new_adopters`)
 - Aggregates: batt_kwh_cum, system_kw_cum, number_of_adopters, medians
 - Faceted plots by state (Baseline vs Policy)
 - EIA price compare

In [ ]:
# Set path to access helpers
from pathlib import Path
import sys

PARENT = Path.cwd().parent     # helpers live here
if str(PARENT) not in sys.path:
    sys.path.insert(0, str(PARENT))

In [ ]:
# Imports
from analysis_functions import (
    SavingsConfig, process_all_states,
    facet_lines_by_state, bar_tech_potential_2040, facet_lines_all_states_delta, facet_lines_national_totals
)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set_style("whitegrid")

# Paths / settings
ROOT_DIR = "/Volumes/Seagate Portabl/permit_power/dgen_runs/per_state_outputs"  # parent with state subfolders
RUN_ID   = "test_run_all_states"   
N_JOBS   = 10                                                                    # use 6–8 on a 12-core machine

# Savings config: 25-year cohort lifetime, do NOT cap to modeling horizon
cfg = SavingsConfig(lifetime_years=25, cap_to_horizon=False)



#### Process all states (parallel)
##### Produces small, tidy DataFrames ready for plotting:
 - result["totals"] → batt_kwh_cum, system_kw_cum, number_of_adopters
 - result["portfolio_annual_savings"], result["cumulative_bill_savings"]
 - result["median_system_kw"], result["tech_2040"], result["lifetime_totals"], ...



In [ ]:
result = process_all_states(ROOT_DIR, run_id=RUN_ID, cfg=cfg, n_jobs=N_JOBS, states=['NY', 'FL', 'CA', 'TX'])
for k, df in result.items():
    print(f"{k:28s}  rows={len(df):,}")

#### Faceted comparisons by state (Baseline vs Policy)

In [ ]:
# 1) Cumulative storage (kWh)
facet_lines_by_state(
    result["totals"], 
    y_col="batt_kwh_cum",
    ylabel="Cumulative Battery (kWh)",
    title="Battery Deployment (Cumulative)"
)

# 2) Cumulative PV (kW)
facet_lines_by_state(
    result["totals"],
    y_col="system_kw_cum",
    ylabel="Cumulative PV (kW)",
    title="PV Deployment (Cumulative)"
)

# 3) Cumulative adopters (count)
facet_lines_by_state(
    result["totals"],
    y_col="number_of_adopters",
    ylabel="Cumulative Adopters",
    title="Adopters (Cumulative)"
)

# 4) Annual portfolio bill savings (cohort carry-forward)
facet_lines_by_state(
    result["portfolio_annual_savings"],
    y_col="portfolio_annual_savings",
    ylabel="Annual Portfolio\nBill Savings ($)",
    title="Annual Portfolio Bill Savings (Carry-forward)"
)

# 5) Cumulative bill savings through year
facet_lines_by_state(
    result["cumulative_bill_savings"],
    y_col="cumulative_bill_savings",
    ylabel="Cumulative Bill Savings ($)",
    title="Cumulative Bill Savings (All Cohorts)"
)

# 6) Technical potential reached in 2040 (bar, sorted by Policy %)
bar_tech_potential_2040(result["tech_2040"])

# 7) Median PV size by state
facet_lines_by_state(
    result["median_system_kw"],
    y_col="median_system_kw",
    ylabel="Median System Size (kW)",
    title="Median PV System Size by Scenario",
)

# 8) Average prices by state comparing model to EIA

avg_prices = pd.read_csv("../../../data/average_retail_elec_price.csv")

if not result["avg_price_2026_model"].empty:
    try:
        price_cmp = (result["avg_price_2026_model"]
                     .merge(avg_prices, on="state_abbr", how="inner"))
        price_cmp["avg_elec_price_cents_per_kwh"] *= 100.0

        # Long-form for grouped bars
        price_melted = price_cmp.melt(
            id_vars="state_abbr",
            value_vars=["avg_elec_price_cents_per_kwh", "cents_per_kwh"],
            var_name="Source", value_name="Average Price (¢/kWh)"
        ).replace({
            "avg_elec_price_cents_per_kwh": "Model",
            "cents_per_kwh": "EIA"
        })

        # Sort by model price
        order = (price_melted[price_melted["Source"] == "Model"]
                 .sort_values("Average Price (¢/kWh)", ascending=False)
                 ["state_abbr"].tolist())

        plt.figure(figsize=(14,6))
        ax = sns.barplot(
            data=price_melted, x="state_abbr", y="Average Price (¢/kWh)",
            hue="Source", order=order, errorbar=None
        )
        for container in ax.containers:
            for bar in container:
                h = bar.get_height()
                if h > 0:
                    ax.text(bar.get_x()+bar.get_width()/2, h-1.5, f"{h:.1f}",
                            ha="center", va="top", color="white", fontsize=9, fontweight="bold")
        plt.title("Average Electricity Price in 2026: Model vs EIA (Sorted by Model)")
        plt.ylabel("¢/kWh"); plt.xlabel("State"); plt.xticks(rotation=45); plt.tight_layout(); plt.show()
    except NameError:
        print("avg_prices not defined; skip EIA comparison.")

In [ ]:
# Faceted national deltas for all metrics:
facet_lines_all_states_delta(result)

# faceted national totals for all metrics
facet_lines_national_totals(result)